In [2]:
import pennylane as qml
from pennylane import numpy as np
print("done")

done


In [4]:
# a device (abstraction) is simply anything that can apply quantum operations and return an expectation value
dev = qml.device("default.qubit", wires=1)
# wires param is the number of subsystems (?) to initialize device with

In [11]:
# a quantum node is an abstract encapsulation of a quantum function, described by a quantum circuit
# bound to a particular devide
@qml.qnode(dev)
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

In [21]:
# this is now a quantum circuit which can be run on the device... i.e. to evaluate
circuit([np.random.randn(),np.random.randn()])

0.11554036614851931

In [30]:
# so we can simply apply AD to get gradients of the function - i.e. get grad()
# function which returns the gradient as a function!
dcircuit = qml.grad(circuit,argnum=0)
# we can then evaluate this function at any(!) point in parameter space
dcircuit([0.54, 0.12])

[-0.510438652516502, -0.10267819945693203]

In [31]:
# we can do standard optimization based on standard optimizers... adam etc...
# define cost function... i.e. our outcome is pauliZ xpectation with expectation value of 01
# we know that it is bound so cost is simply qnode output
def cost(param):
    return circuit(param)



In [33]:
# let's choose small initial values and then sort it out
init_params = np.array([0.011, 0.012])
cost(init_params)

0.9998675058299387

In [34]:
# then do gradient descent updates on this... which is AWESOME!
opt = qml.GradientDescentOptimizer(stepsize=0.4)
steps = 100
params = init_params
for i in range(steps):
    params = opt.step(cost, params)
    if (i+1) % 5 == 0:
        print('Cost after step {:5d}: {: .7f}'.format(i+1, cost(params)))

print('Optimized rotation angles: {}'.format(params))

Cost after step     5:  0.9961778
Cost after step    10:  0.8974944
Cost after step    15:  0.1440490
Cost after step    20: -0.1536720
Cost after step    25: -0.9152496
Cost after step    30: -0.9994046
Cost after step    35: -0.9999964
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [5.76516144e-17 3.14159265e+00]


In [ ]:
# it works!! we have optimized a quantum circuit... this is awesome!!!